In [3]:
from __future__ import division, absolute_import, print_function
import sys
import os
from pylab import *  # for plotting
from graph_tool.all import *
import random as rd
import itertools #Biblioteca para geração da lista de elementos repetidos
import numpy as np

## Número de sítios da rede

Logo de começo devemos restringir o valor de $m$ com relação a $m_0$, fazendo

<center> $ m \le m_0 $, (1) </center>

essa restrição evita ligações em loop do novo sítio. Nós temos que o número de sítos final da rede será dada por

<center> $N = m_0 + t$, (2) </center>

para cada instante de tempo $t$, haverá $m_0+t$ sítios. Com número de ligações dadas por

<center> $l = m_0+mt$, (3)</center>

pois cada um dos $t$ sítios terão $m$ ligações. Para sabermos quanto devemos utilizar no tempo para um número $N$ de sítios (quantas interações devemos fazer), podemos isolar o $t$ em (2),obtendo

<font size="3"><center> $N-m_0 = t$.  </center></font>

A probabilidade de ligação de um novo sítio $i$ é dada por

<font size="3"><center> $p_i = \frac{k_i}{\sum_i k_i}$ </center></font>,

com a distribuição dos graus dados por

<font size="3"><center> p(k) = $\frac{2m_0(m_0+1)}{k(k+1)(k+2)}$ </center></font>



## 2. Algoritmo para rede inicial

Tomando a rede inicial com $m_0$ sítios totalmente conectada (sítios igualmente provaveis inicialmente), teremos a matriz de adjacência dada por

\begin{equation}
A = 
\begin{pmatrix}
0 & 1 & 1 & \cdots & 1\\
1 & 0 & 1 & \cdots & 1\\
1 & 1 & 0 & \cdots & 1\\
1 & 1 & 1 & \ddots & 1\\
1 & 1 & 1 & \cdots & 0\\
\end{pmatrix}
_{(m_0,m_0)},
\end{equation}

A rede totalmente conectada, possuí todos elementos fora da coluna principal igual a 1. Para isso, podemos expressar um algoritmo primeiro fixando não-ligação para $i=j$ (sem loops), e logo em seguinda tomando a condição de que $j\le i$, para pegar apenas os elementos do triângulo superior da matriz.

In [23]:
g = Graph(directed=False)

m0 = 4  #Número de sítios iniciais;
m = 3  #Número de conexões dos novos sítios;
N = 10 #Número de sítios final;
t = N - m0 #Número de passos, lembrando que após t instantes de tempo, teremos N = m0 + t sítios;

#Construindo a rede inicial------------------------------------------

g.add_vertex(m0) #Add m0 elements de sítios em g;

#Construindo uma rede inicial totalmente conectada
for i in range(m0):
    for j in range(i+1,m0):
        g.add_edge(g.vertex(i),g.vertex(j))

K = [] #Lista que guardará os "k" valores dos "i" sítios;
K_supli = [] #Lista Suplementar, que juntará os novos elementos ao K;
K_filter = [] #Lista que será filtrada para ser usada dentro do loop;

#Construindo K e K_filter----------------------------------------------
for i in range(m0):
    K_supli = list(itertools.repeat(int(g.vertex(i)), int(g.vertex(i).out_degree())))
    K += K_supli
    K_filter += K_supli

print(K)
print(g.get_vertices())

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3]
[0 1 2 3]


In [24]:
m0 = 4  #Número de sítios iniciais;
m = 3  #Número de conexões dos novos sítios;
N = 10

for i in range(m0,N):
    v = g.add_vertex()
    for j in range(m):
        j = randint(0, len(K_filter)-1)
        u = g.vertex(K_filter[j])
        L = g.add_edge(v,u)
        K += [int(u)]
        print(len(K_filter))
        K_filter = list(filter(lambda val: val !=  int(u), K_filter))
        
        
    K_supli = list(itertools.repeat(int(g.vertex(i)), int(g.vertex(i).out_degree())))
    K += K_supli
    K_filter = K
print(K_filter)

12
9
6
19
14
10
25
19
15
31
26
20
37
31
26
43
36
30
[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0, 1, 3, 4, 4, 4, 3, 0, 2, 5, 5, 5, 3, 1, 0, 6, 6, 6, 2, 0, 3, 7, 7, 7, 1, 2, 3, 8, 8, 8, 2, 1, 3, 9, 9, 9]


In [ ]:
q = Graph()
m = 3
K_t = [0,0,0,1,1,1,3,3,3,4,4,4]

for i in range(1,3):
    i = randint(0,len(K_t))
    g.vertex()

In [66]:
for i in range(3):
    j = randint(0, len(K_filter)-1)
    u = g.vertex(K_filter[j])
    L = g.add_edge(v,u)
    K_filter = list(filter(lambda val: val !=  u, K_filter))
    print(j)
    print(K_filter)
K_filter = K



3
[0, 0, 0, 2, 2, 2, 3, 3, 3]
4
[0, 0, 0, 3, 3, 3]
2
[3, 3, 3]


In [9]:
a = 2

In [14]:
#m0: número de sítios' iniciais;
def barabasi_network(m0,m,N,s):
    seed(s)
    g = Graph(directed=False) #Grafo sem direção
    
    #S = g.new_vertex_property("double")
    #L = g.new_edge_property("double")
    
    g.add_vertex(m0) #Add m0 elements de sítios em g;

    K = [] #Lista que guardará os os valores dos sítios i, k vezes;
    K_supli = [] #Lista Suplementar, que juntará os novos elementos ao K;
    K_filter = [] #Lista que será filtrada para ser usada dentro do loop;
    
    #Criando a rede inicial----------------------------------------------------------
    ver = np.zeros(N) #Guardará os sítios num array;
    
    #Rede inicial totalmente conectada;
    for i in range(m0):
        ver[i] = int(g.vertex(i)) # Definindo todos valores dos sítios i como inteiros
        for j in range(i+1,m0):
            g.add_edge(g.vertex(i),g.vertex(j)) #Construindo uma rede totalmente ligada (inicial)
    
    #Construção da lista K inicial--------------------------------------------------
    for i in range(m0):
        #Gera uma  lista de elementos repetidos
        K_supli = list(itertools.repeat(int(g.vertex(i)), int(g.vertex(i).out_degree()))) 
        
        K += K_supli #Add essa lista de elementos repetidos em K
        K_filter += K_supli #Add essa lista de elementos repetidos em K
        
    #---------------------------------Crescimento da rede-------------------------------
    for i in range(m0,N):
        v = g.add_vertex() #Adiciona um novo sítio
        
        # O for abaixo é responsável por sortear um sítio dentre todos existentes da rede (antes da adição
        # do novo sítio), e realizar uma ligação desse sítio sorteado com o sítio adicionado. Após isso, é
        #feito uma filtragem, deixando apenas os sítios restantes (evitar sítios com duas ligações), essa filtragem
        #é feita m vezes.
        
        for j in range(m):
            Num = randint(0,len(K_filter)-1) #Gera um número aleatório(sítio) dentro de K_filter ;
            u = g.vertex(K_filter[Num]) #seleciona o sítio que foi gerado acima (transforma int em vertex);
            L = g.add_edge(v,g.vertex(K_filter[Num])) #Realiza a ligação do sítio add como o sítio sorteado;
            K += [int(u)] #Adiciona esse sítio sorteado aos valores de K;
            #Filtra K, removendo o sítio já sorteado;
            K_filter = list(filter(lambda val: val !=  int(u), K_filter)) 
        
        #Constroi uma lista com as m ligações do sítio novo (v)
        K_supli = list(itertools.repeat(int(v), int(v.out_degree())))
        K += K_supli #Add a lista acima em K
        #Atualiza K_filter, add os novos sítios (v) e retomando K_filter a forma original (antes da filtragem)
        K_filter = K  
    
    return K

In [19]:
K_t = barabasi_network(5,3,10000,1001)

KeyboardInterrupt: 

In [ ]:
print(K_t)